In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk


  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-4mtq1nrp/gym-walk_75c5d76d6f3e43b2ab4509b353ee0258
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-4mtq1nrp/gym-walk_75c5d76d6f3e43b2ab4509b353ee0258
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=9465d5d62b12dac06c38711cd0cbb7cabfbda7461933ca38b504fbcf2ee36edb
  Stored in directory: /tmp/pip-ephem-wheel-cache-1wggc897/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [2]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")
import warnings ; warnings.filterwarnings('ignore')
from tqdm import tqdm
import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)


In [4]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [5]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [6]:

def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)


In [7]:

env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
#LEFT, RIGHT = range(2)


In [8]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [15]:
def decay_schedule(
    init_value, min_value, decay_ratio,
    max_steps, log_start = -2, log_base=10):

  #Write your code here
   decay_steps = int(max_steps* decay_ratio)
   rem_steps=max_steps-decay_steps
   values=np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]
   values=(values-values.min())
   values=(init_value-min_value)*values+min_value
   values=np.pad(values,(0,rem_steps),'edge')

   return values

In [16]:
from itertools import count
def generate_trajectory(
    select_action, Q, epsilon,
    env, max_steps=200):
  done, trajectory = False, []

  #Write your code here
  done,trajectory=False,[]
  while not done:
    state=env.reset()
    for t in count():
      action=select_action(state,Q,epsilon)
      next_state,reward,done,_=env.step(action)
      experience=(state,action,reward,next_state,done)
      trajectory.append(experience)
      if done:
        break
      if t>=max_steps-1:
        trajectory=[]
        break
      state=next_state
    return np.array(trajectory,object)


In [17]:
!pip install --upgrade numpy

In [18]:
from tqdm import tqdm
def mc_control(env, gamma=1.0,
               init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
               init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
               n_episodes=3000, max_steps=200, first_visit=True):

    ns, na = env.observation_space.n, env.action_space.n
    discounts = np.logspace(0, max_steps, num=max_steps, base=gamma, endpoint=False)
    alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
    epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

    Q = np.zeros((ns, na), dtype=np.float64)
    Q_track = np.zeros((n_episodes, ns, na), dtype=np.float64)
    pi_track = []

    select_action = lambda state, Q, epsilon: np.argmax(Q[state]) if np.random.random() > epsilon else np.random.randint(len(Q[state]))

    for e in tqdm(range(n_episodes), leave=False):
        trajectory = generate_trajectory(select_action, Q, epsilons[e], env, max_steps)
        visited = np.zeros((ns, na), dtype=bool)

        for t, (state, action, reward, _, _) in enumerate(trajectory):
            if visited[state][action] and first_visit:
                continue
            visited[state][action] = True
            n_steps = len(trajectory[t:])
            G = np.sum(discounts[:n_steps] * np.array([x[2] for x in trajectory[t:]]))
            Q[state][action] += alphas[e] * (G - Q[state][action])

        Q_track[e] = Q
        pi_track.append(np.argmax(Q, axis=1))

    V = np.max(Q, axis=1)
    pi = lambda s: np.argmax(Q[s])
    return Q, V, pi, Q_track, pi_track

In [19]:
optimal_Q, optimal_V, optimal_pi,_,_= mc_control (env,n_episodes=3000)
print('Name: THARUN D     Register Number:212223240167')
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)

Name: THARUN D     Register Number:212223240167
Action-value function:
| 00 [0.26 0.17 0.15 0.13] | 01 [0.08 0.04 0.02 0.1 ] | 02 [0.06 0.05 0.09 0.02] | 03 [0.02 0.03 0.02 0.1 ] |
| 04 [0.27 0.1  0.14 0.08] |           | 06 [0.04 0.1  0.01 0.  ] |           |
| 08 [0.12 0.1  0.08 0.29] | 09 [0.12 0.34 0.17 0.13] | 10 [0.24 0.4  0.13 0.08] |           |
|           | 13 [0.06 0.08 0.15 0.33] | 14 [0.16 0.49 0.71 0.3 ] |           |
State-value function:
| 00   0.26 | 01    0.1 | 02   0.09 | 03    0.1 |
| 04   0.27 |           | 06    0.1 |           |
| 08   0.29 | 09   0.34 | 10    0.4 |           |
|           | 13   0.33 | 14   0.71 |           |
Policy:
| 00      < | 01      ^ | 02      > | 03      ^ |
| 04      < |           | 06      v |           |
| 08      ^ | 09      v | 10      v |           |
|           | 13      ^ | 14      > |           |


In [20]:
# Find the probability of success and the mean return of you your policy
print('Name:THARUN D    Register Number:212223240167')
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100,
    mean_return(env, optimal_pi)))

Name:THARUN D    Register Number:212223240167
Reaches goal 40.00%. Obtains an average undiscounted return of 0.4000.
